In [1]:
import pandas as pd
import numpy as np


from data_loader import load_from_tsfile_to_dataframe
from regressor_tools import process_data
import mlflow
from tsfeatures import tsfeatures

In [ ]:
# DCT or DFT Compression
# precise cutting and not with threshold_value

#->> both methods should be alright, but need to be field tested!

import numpy as np
from scipy.fft import dct, idct, fft, ifft

def dct_or_dft_compress(signal, dropout_ratio, transform_type='dft'):
    if transform_type == 'dct':
        # Apply DCT to the signal
        signal_transformed = dct(signal, type=2, norm='ortho')
        inverse_transform = idct
    elif transform_type == 'dft':
        # Apply FFT to the signal
        signal_transformed = fft(signal)
        inverse_transform = ifft
    else:
        raise ValueError("Invalid transform_type. Choose 'dct' or 'dft'.")
    
    # Calculate the number of coefficients to keep
    num_coeffs = int((1 - dropout_ratio) * len(signal_transformed))
    
    # Sort the coefficients by magnitude and keep only the top ones
    sorted_indices = np.argsort(np.abs(signal_transformed))
    kept_indices = sorted_indices[-num_coeffs:]
    
    # Set the coefficients to zero for the dropped out indices
    compressed_transform = np.zeros_like(signal_transformed)
    compressed_transform[kept_indices] = signal_transformed[kept_indices]
    
    # Reconstruct the signal using inverse transform
    compressed_signal = inverse_transform(compressed_transform)
    
    return compressed_signal